1. Tag p10 parcels by if falling into Communities of Concern (convert FileGDB to .csv)
2. Tag p10 parcels by if residential building is present on the parcel

In [1]:
import fiona
import geopandas as gpd
import pandas as pd

### p10 parcel-coc tagging

In [2]:
f = (r'C:\Users\ywang\Documents\ArcGIS\Projects\EIR_zoningmods\EIR_zoningmods.gdb')
layers = fiona.listlayers(f)
display(layers)

df =  gpd.read_file(f,
                     driver='FileGDB',
                     layer='p10_pba50_coc_tbl_v2_02172021')
print('read {} rows from parcel_coc tagging file'.format(df.shape[0]))

p10_pba50_cocid = df[['PARCEL_ID', 'coc_id']]
p10_pba50_cocid.loc[p10_pba50_cocid.coc_id != 'CoC', 'coc_id'] = 'NA'

p10_pba50_cocid['PARCEL_ID'] = p10_pba50_cocid['PARCEL_ID'].apply(lambda x: int(round(x)))

print('{} parcels fall into CoC'.format(p10_pba50_cocid.loc[p10_pba50_cocid.coc_id == 'CoC'].shape[0]))

# export 
p10_pba50_cocid.to_csv('C:\\Users\\ywang\\Box\\Modeling and Surveys\\Urban Modeling\\Bay Area UrbanSim\\PBA50\\Policies\\Zoning Modifications\\p10_pba50_cocid.csv', index=False)

['CoC_only_2020_acs2018',
 'p10_pba50_CoC',
 'p10_pba50_CoC_2020_acs2018',
 'p10_pba50_CoC_2020_acs2018_tbl',
 'p10_pba50_coc_tbl_v1_02122021',
 'p10_pba50_coc_tbl_v2_02172021']

read 1956208 rows from parcel_coc tagging file


C:\Users\ywang\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\ywang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


299664 parcels fall into CoC


### p10 existing residential present

In [3]:
# read basemap p10 and b10 data
p10 = pd.read_csv(r'M:\Data\GIS layers\UrbanSim smelt\2020 03 12\p10.csv')
b10 = pd.read_csv(r'M:\Data\GIS layers\UrbanSim smelt\2020 03 12\b10.csv')

# make sure PARCEL_ID is integer
p10['PARCEL_ID'] = p10['PARCEL_ID'].apply(lambda x: int(round(x)))
print('Read {} rows from p10, with {} unique PARCEL_IDs'.format(p10.shape[0], len(p10.PARCEL_ID.unique())))

b10['parcel_id'] = b10['parcel_id'].apply(lambda x: int(round(x)))
print('Read {} rows from b10, with {} unique PARCEL_IDs'.format(b10.shape[0], len(b10.parcel_id.unique())))

C:\Users\ywang\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Read 1956208 rows from p10, with 1956208 unique PARCEL_IDs
Read 1843351 rows from b10, with 1843292 unique PARCEL_IDs


In [4]:
# merge parcels to buildings 
building_parcel = b10.merge(p10[['PARCEL_ID','LAND_VALUE','ACRES']],
                            left_on='parcel_id', 
                            right_on='PARCEL_ID', 
                            how='outer')

# aggregate building data by parcel
building_groupby_parcel = building_parcel.groupby(['PARCEL_ID']).agg({
    'improvement_value'   :'sum',
    'residential_units'   :'sum',
    'residential_sqft'    :'sum',
    'non_residential_sqft':'sum',
    'building_sqft'       :'sum',
    'year_built'          :'min',
    'building_id'         :'min'}).reset_index()


# tag 
building_groupby_parcel['ex_res_bldg'] = 'NA'

has_res1 = building_groupby_parcel.loc[building_groupby_parcel.residential_units > 0]
print('{} parcels have positive residential_units'.format(has_res1.shape[0]))

has_res2 = building_groupby_parcel.loc[building_groupby_parcel.residential_sqft > 0]
print('{} parcels have positive residential_sqft'.format(has_res2.shape[0]))

building_groupby_parcel.loc[building_groupby_parcel.residential_sqft > 0, 'ex_res_bldg'] = 'res'

1696059 parcels have positive residential_units
1696008 parcels have positive residential_sqft


In [5]:
# export

print('export {} rows'.format(building_groupby_parcel.shape[0]))

building_groupby_parcel[['PARCEL_ID', 'ex_res_bldg']].to_csv('C:\\Users\\ywang\\Box\\Modeling and Surveys\\Urban Modeling\\Bay Area UrbanSim\\PBA50\\Policies\\Zoning Modifications\\p10_existing_res_bldg.csv', index=False)

export 1956208 rows
